### 1. Section on outlier removal 
- created this section after finding non-normal errors
- but the errors really aren't too far from normal
- and also even after removing outliers the Shapiro-Wilk test failed

In this example, we used the Isolation Forest to detect outliers and marked them in the DataFrame using an additional column called 'outlier'. The contamination parameter determines the proportion of outliers expected in the data (typically set between 0 and 0.5). You can adjust this parameter based on your domain knowledge or perform cross-validation to find an optimal value.

After outlier detection, you can further analyze the 'outlier' column to investigate the outliers and decide whether to remove them from the dataset. Keep in mind that outlier removal should be done judiciously, considering the context and domain knowledge, and it may depend on the size and nature of your dataset.


In [2]:
from sklearn.ensemble import IsolationForest

# Combine the target and predictor columns
all_data = pd.concat([y_log, X], axis=1)
contamination = 0.05
# Perform outlier detection using Isolation Forest
outlier_detector = IsolationForest(contamination=contamination)
outliers = outlier_detector.fit_predict(all_data)
# Add an 'outlier' column to mark the outliers
all_data['outlier'] = np.where(outliers == -1, True, False)
# Reprot number of outliers
all_data['outlier'].sum()

# Separate the cleaned data and target variable
X_clean = all_data.drop(columns=['outlier', y_log.name])
y_log_clean = all_data[y_log.name]

# Remove rows with outliers from the data
X_clean = X_clean[~all_data['outlier']].copy()
y_log_clean = y_log_clean[~all_data['outlier']].copy()

NameError: name 'pd' is not defined

NameError: name 'all_data' is not defined

#### Retrain model after performing outlier detection

In [ ]:
# Retrain on clean data
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_log_clean, 
                                                    test_size=0.33, 
                                                    random_state=4)

# Add a constant column to the predictor variables dataframe
X_train = sm.add_constant(X_train)

# Add the constant to the test set as well so we can use the model to form predictions on the test set later
X_test = sm.add_constant(X_test)
X_test.head()

# Fit the multivariate regression model
model = sm.OLS(y_train, X_train)
trained_model = model.fit()

# to calculate residuals and R-squared for the test set, we'll need to get the model predictions first
y_pred_test = trained_model.predict(X_test)

# calculate residuals 
test_residuals = y_pred_test - y_test

In [ ]:
from helper_functions import assess_normal_resid 

assess_normal_resid(trained_model, test_residuals)

### 2. Section on comparing real data QQ-plot to simulated data QQ-plot; decided this section probably isn't worth the time

What happens if we decrease the sample size of our simulation? Let's try to generate a QQ-plot containing only 200 samples.

In [ ]:
np.random.seed(0) # set seed for reproducibility
normal_data = np.random.normal(loc=0, scale=1, size=200) # mean of 0, std of 1, len(samples) = number of observations used to train/fit model
normal_data.shape

## create qq-plot
import statsmodels.api as sm
sm.qqplot(normal_data, line='s');

# Shapiro-Wilk test for normality
shapiro_stat, shapiro_p = stats.shapiro(normal_data)
print(f"Shapiro-Wilk test: statistic={shapiro_stat:.4f}, p-value={shapiro_p:.10f}")

In [ ]:
Notice how the SW test nearly fails even though this data comes from a normal distrition. Testing normality can be a difficult task when data is very limited. What happens if we decrease the sample size of our simulation? Let's try to generate a QQ-plot containing the same number of samples used to train our model.